In [ ]:
!pip install datasets
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio
!pip install accelerate -U
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 13.1 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-550lzmqn
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-550lzmqn
  Resolved https://github.com/huggingface/transformers to commit 483861d52db59cf99219a0281695d1e7e8859218
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.6 M

In [ ]:
!unzip /content/Hindi_Commonvoice_data-20230816T105523Z-001.zip

Archive:  /content/Hindi_Commonvoice_data-20230816T105523Z-001.zip
  inflating: Hindi_Commonvoice_data/data/val/texts/common_voice_hi_37873511.txt  
  inflating: Hindi_Commonvoice_data/data/val/texts/common_voice_hi_37754363.txt  
  inflating: Hindi_Commonvoice_data/data/val/texts/common_voice_hi_37754360.txt  
  inflating: Hindi_Commonvoice_data/data/val/texts/common_voice_hi_37895003.txt  
  inflating: Hindi_Commonvoice_data/data/val/texts/common_voice_hi_37890679.txt  
  inflating: Hindi_Commonvoice_data/data/val/texts/common_voice_hi_37873512.txt  
  inflating: Hindi_Commonvoice_data/data/val/texts/common_voice_hi_37817520.txt  
  inflating: Hindi_Commonvoice_data/data/val/texts/common_voice_hi_37817524.txt  
  inflating: Hindi_Commonvoice_data/data/val/texts/common_voice_hi_37890680.txt  
  inflating: Hindi_Commonvoice_data/data/val/texts/common_voice_hi_37890682.txt  
  inflating: Hindi_Commonvoice_data/data/val/texts/common_voice_hi_37803369.txt  
  inflating: Hindi_Commonvoice_

In [ ]:
import glob
import re
path = glob.glob('/content/Hindi_Commonvoice_data/data/train/texts/*.txt')
audio_path = '/content/Hindi_Commonvoice_data/data/train/audios'

In [ ]:
path

In [ ]:
data=[]
with open('train.csv', 'w') as g:
  g.write(f"text,path,audio\n")
  for paths in path:
    name = paths.split('/')[-1]
    name_audio = name.split('.')[0] + '.mp3'
    with open(paths, 'r') as f:
      for file in f:
        text = file.strip()
        res = re.sub(r'[^\w\s]', '', text)
        g.write(f"{res},{audio_path},{audio_path}/{str(name_audio)}\n")

In [ ]:
!cat train.csv

text,path,audio


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from datasets import Dataset, Audio, Value, Features
from datasets import load_dataset

features = Features(
    {
        "text": Value("string"),
        'path': Value('string'),
        "audio": Audio(sampling_rate=16000)
    }
)
sample_data = load_dataset(
    'csv', data_files={
        'train': 'train.csv',
        'valid': 'valid.csv'
    }
)

#import pandas as pd
sample_data = sample_data.cast(features)
#pd.read_csv("/content/train.csv")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating valid split: 0 examples [00:00, ? examples/s]

Casting the dataset:   0%|          | 0/149 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/33 [00:00<?, ? examples/s]

In [ ]:
from transformers import WhisperFeatureExtractor
from transformers import WhisperTokenizer
from transformers import WhisperProcessor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-tiny")
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-tiny", language="Assamese", task="transcribe")
processor = WhisperProcessor.from_pretrained("openai/whisper-tiny", language="Assamese", task="transcribe")

In [ ]:
print(sample_data["train"][0])

{'text': 'दलल लफट क बहन कर लट क कशश डरइवर क मर गल', 'path': '/content/Hindi_Commonvoice_data/data/train/audios', 'audio': {'path': '/content/Hindi_Commonvoice_data/data/train/audios/common_voice_hi_37579874.mp3', 'array': array([ 5.82076609e-11, -2.91038305e-11,  1.16415322e-10, ...,
        1.03300090e-06, -2.21371670e-06, -2.24964128e-06]), 'sampling_rate': 16000}}


In [ ]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["text"]).input_ids
    return batch

In [ ]:
custom_data = sample_data.map(prepare_dataset, remove_columns=sample_data.column_names["train"], num_proc=2)

Map (num_proc=2):   0%|          | 0/149 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/33 [00:00<?, ? examples/s]

In [ ]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [ ]:
import evaluate

metric = evaluate.load("wer")

In [ ]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny")

In [ ]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="/content/whisper-tiny-as",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=100,
    max_steps=2000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=100,
    eval_steps=100,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
)

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=custom_data["train"],
    eval_dataset=custom_data["valid"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

In [ ]:
processor.save_pretrained(training_args.output_dir)

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss


NameError: ignored